In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from math import sin, cos, sqrt, atan2, radians
from mpl_toolkits.mplot3d import Axes3D

In [2]:
df = pd.read_pickle('boosted_dataset.pkl')

In [15]:
def haversine_to_downtown(point):
    # calculates the distance between two points (lat, lngs) on a great circle, or on the 
    # surface of a sphere (in this case the sphere is planet earth)
    # units in km
    lat, lng = point
    deglen = 110.25
    x = lat - 30.2648
    y = (lng - (-97.7472))*cos(-97.7472)
    return deglen*sqrt(x*x + y*y)

In [12]:
# get list of coordinates
subset = df[['lat', 'lon']]
tuples = [tuple(x) for x in subset.values]

In [7]:
distances = 

c_street_number                                       15302
c_street                                      DOROTHY DRIVE
c_unit                                                 None
c_city                                               AUSTIN
c_state                                                  TX
c_zip_code                                            78734
g_county                                      Travis County
lat                                                 30.3528
lon                                                -97.9568
adj_address              15302 DOROTHY DRIVEAUSTIN TX 78734
price_per_sqft                                      175.622
price_per_sqft_gboost                               194.868
price                                                609196
sqft                                                 3468.8
bathrooms                                           3.77397
bedrooms                                                  4
Name: 3, dtype: object

In [14]:
haversine_to_downtown((30.3528,-97.9568))

23.71966045924804